In [19]:
import pandas as pd
import numpy as np

In [20]:
### 참고사항 ###
# y_test는 실기 문제상에 주어지지 않음

# Tip : X를 대문자로 쓰지말고 소문자 x로 쓰자. 시험에서 실수하기 쉽다. ( 문제풀기 전에 소문자로 변경! )
# (참고 : 보통 X는 2차원 배열(행렬)이기 때문에 대문자로 쓰고, y는 1차원 배열(벡터)이기 때문에 소문자로 씀)

# ( 참고 ) 실기시험 데이터 형식 ( 실제 시험장에서는 다를 수 있으니 반드시 체크 )
# X_test = pd.read_csv('data/X_test.csv')
# X_train = pd.read_csv('data/X_train.csv')
# y_train = pd.read_csv('data/y_train.csv')

### 와인의 종류를 분류해보자

In [21]:
from sklearn.datasets import load_wine
# 와인 데이터셋을 로드합니다.
wine = load_wine()
x = pd.DataFrame(wine.data, columns=wine.feature_names)
y = pd.DataFrame(wine.target)

# 실기 시험 데이터셋으로 셋팅하기 (수정금지)
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, 
                                                    stratify=y,
                                                    random_state=2023)

x_test = pd.DataFrame(x_test)
x_train = pd.DataFrame(x_train)
y_train = pd.DataFrame(y_train)

x_test.reset_index()
y_train.columns = ['target'] 

- 데이터의 결측치, 이상치에 대해 처리하고
- 분류모델을 사용하여 정확도, F1 score, AUC 값을 산출하시오.
- 제출은 result 변수에 담아 양식에 맞게 제출하시오

In [22]:
# 데이터 설명
print(wine.DESCR)

.. _wine_dataset:

Wine recognition dataset
------------------------

**Data Set Characteristics:**

:Number of Instances: 178
:Number of Attributes: 13 numeric, predictive attributes and the class
:Attribute Information:
    - Alcohol
    - Malic acid
    - Ash
    - Alcalinity of ash
    - Magnesium
    - Total phenols
    - Flavanoids
    - Nonflavanoid phenols
    - Proanthocyanins
    - Color intensity
    - Hue
    - OD280/OD315 of diluted wines
    - Proline
    - class:
        - class_0
        - class_1
        - class_2

:Summary Statistics:

============================= ==== ===== ======= =====
                                Min   Max   Mean     SD
============================= ==== ===== ======= =====
Alcohol:                      11.0  14.8    13.0   0.8
Malic Acid:                   0.74  5.80    2.34  1.12
Ash:                          1.36  3.23    2.36  0.27
Alcalinity of Ash:            10.6  30.0    19.5   3.3
Magnesium:                    70.0 162.0    99.7  14.3

### 2. 데이터 탐색(EDA)

In [23]:
# 데이터의 행/열 확인

print(x_train.shape)
print(x_test.shape)
print(y_train.shape)

(142, 13)
(36, 13)
(142, 1)


In [24]:
# 초기 데이터 확인

print(x_train.head(3))
print(x_test.head(3))
print(y_train.head(3))

     alcohol  malic_acid   ash  alcalinity_of_ash  magnesium  total_phenols  \
52     13.82        1.75  2.42               14.0      111.0           3.88   
146    13.88        5.04  2.23               20.0       80.0           0.98   
44     13.05        1.77  2.10               17.0      107.0           3.00   

     flavanoids  nonflavanoid_phenols  proanthocyanins  color_intensity   hue  \
52         3.74                  0.32             1.87             7.05  1.01   
146        0.34                  0.40             0.68             4.90  0.58   
44         3.00                  0.28             2.03             5.04  0.88   

     od280/od315_of_diluted_wines  proline  
52                           3.26   1190.0  
146                          1.33    415.0  
44                           3.35    885.0  
     alcohol  malic_acid   ash  alcalinity_of_ash  magnesium  total_phenols  \
168    13.58        2.58  2.69               24.5      105.0           1.55   
144    12.25        

In [25]:
# 변수명과 데이터 타입이 매칭이 되는지, 결측치가 있는지 확인하자.

print(x_train.info())
print(x_test.info())
print(y_train.info())

<class 'pandas.core.frame.DataFrame'>
Index: 142 entries, 52 to 115
Data columns (total 13 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   alcohol                       142 non-null    float64
 1   malic_acid                    142 non-null    float64
 2   ash                           142 non-null    float64
 3   alcalinity_of_ash             142 non-null    float64
 4   magnesium                     142 non-null    float64
 5   total_phenols                 142 non-null    float64
 6   flavanoids                    142 non-null    float64
 7   nonflavanoid_phenols          142 non-null    float64
 8   proanthocyanins               142 non-null    float64
 9   color_intensity               142 non-null    float64
 10  hue                           142 non-null    float64
 11  od280/od315_of_diluted_wines  142 non-null    float64
 12  proline                       142 non-null    float64
dtypes: float6

In [26]:
# x_train과 x_test 데이터의 기초 통계량을 비교해보자
print(x_train.describe().T, '\n') # x_train.describe().T 둘중에 편한거 사용하세요
print(x_test.describe().T, '\n')
print(y_train.describe().T)

                              count        mean         std     min       25%  \
alcohol                       142.0   13.025915    0.812423   11.03   12.3700   
malic_acid                    142.0    2.354296    1.142722    0.74    1.6100   
ash                           142.0    2.340211    0.279910    1.36    2.1900   
alcalinity_of_ash             142.0   19.354225    3.476825   10.60   16.8000   
magnesium                     142.0   98.732394   13.581859   70.00   88.0000   
total_phenols                 142.0    2.303592    0.633955    0.98    1.7575   
flavanoids                    142.0    2.043592    1.033597    0.34    1.2275   
nonflavanoid_phenols          142.0    0.361479    0.124627    0.14    0.2700   
proanthocyanins               142.0    1.575070    0.576798    0.41    1.2425   
color_intensity               142.0    5.005070    2.150186    1.28    3.3000   
hue                           142.0    0.950394    0.220736    0.54    0.7825   
od280/od315_of_diluted_wines

In [27]:
# y데이터도 구체적으로 살펴보세요.
print(y_train.head())

     target
52        0
146       2
44        0
67        1
43        0


In [28]:
print(y_train.value_counts())

target
1         57
0         47
2         38
Name: count, dtype: int64


### 3. 데이터 전처리 및 분리

#### 1) 결측치, 2) 이상치, 3) 변수 처리하기

In [29]:
# 결측치 확인
print(x_train.isnull().sum(), '\n')
print(x_test.isnull().sum(), '\n')
print(y_train.isnull().sum())

alcohol                         0
malic_acid                      0
ash                             0
alcalinity_of_ash               0
magnesium                       0
total_phenols                   0
flavanoids                      0
nonflavanoid_phenols            0
proanthocyanins                 0
color_intensity                 0
hue                             0
od280/od315_of_diluted_wines    0
proline                         0
dtype: int64 

alcohol                         0
malic_acid                      0
ash                             0
alcalinity_of_ash               0
magnesium                       0
total_phenols                   0
flavanoids                      0
nonflavanoid_phenols            0
proanthocyanins                 0
color_intensity                 0
hue                             0
od280/od315_of_diluted_wines    0
proline                         0
dtype: int64 

target    0
dtype: int64


In [30]:
# 결측치 제거
# df = df.dropna()
# print(df)

# 참고사항
# print(df.dropna().shpae) # 행 기준으로 삭제

In [31]:
# 결측치 대체(평균값, 중앙값, 최빈값)
# 연속형 변수 : 중앙값, 평균값
# - df['변수명'].median()
# - df['변수명'].mean()
# 범주형 변수 : 최빈값

# df['변수명'] = df['변수명'].fillna(대체할 값)

In [32]:
# 이상치 대체(예시)
# df['변수명'] = np.where( df['변수명'] >= 5, 대체할 값, df['변수명'] )

In [33]:
# 변수 처리

# 불필요한 변수 제거
# df = df.drop(columns = ['변수1', '변수2'])

# 필요시 변수 추가(파생변수 생성)
# df['파생변수명'] = df['A'] * df['B'] ( 파생변수 생성 수식 )

# 원 핫 인코딩 ( 가변수 처리 )
# x_train = pd.get_dummies(x_train)
# x_test = pd.get_dummies(x_test)
# print(x_train.info())
# print(x_test.info())

### 데이터 분리

In [34]:
# 데이터를 훈련 세트와 검증용 세트로 분할 ( 80% 훈련, 20% 검증 )
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x_train, 
                                                  y_train['target'], 
                                                  test_size = 0.2,
                                                  stratify = y_train['target'],
                                                  random_state = 2023)

print(x_train.shape)
print(x_val.shape)
print(y_train.shape)
print(y_val.shape)

(113, 13)
(29, 13)
(113,)
(29,)


#### 4. 모델링 및 성능평가

In [35]:
# 랜덤 포레스트 모델 사용 ( 참고 : 회귀모델은 RandomForestRegressor )
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
model.fit(x_train, y_train)

RandomForestClassifier()

In [36]:
# 모델을 사용하여 테스트 데이터 예측
y_pred = model.predict(x_val)

In [40]:
# 모델 성능 평가 ( 정확도, F1 score, 민감도, 특이도 등 )
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score
acc = accuracy_score(y_val, y_pred)             # (실제값, 예측값 )
f1 = f1_score(y_val, y_pred, average = 'macro') # (실제값, 예측값 )
# auc = roc_auc_score(y_val, y_pred)            # (실제값, 예측값 ) * AUC는 이진분류

In [41]:
print(acc)

1.0


In [42]:
print(f1)

1.0


#### 5. 예측값 제출

(주의) test셋을 모델에 넣어 나온 예측값을 제출해야함

In [46]:
# (실기시험 안내사항)
# 아래 코드 예측변수와 수험번호를 개인별로 변경하여 활용
# pd.DataFrame({ 'result': y_result }).to_csv('수험번호.csv', index=False)

# 모델을 사용하여 테스트 데이터 예측

# 1. 특정 클래스로 분류할 경우 (predict)
y_result = model.predict(x_test)
print(y_result[:5])

# 2. 특정 클래스로 분류 될 확률을 구할 경우 (predict_proba)
y_result_prob = model.predict_proba(x_test)
print(y_result_prob[:5])

# 이해해보기
result_prob = pd.DataFrame({
    'result': y_result,
    'prob_0' : y_result_prob[:,0],
    'prob_1' : y_result_prob[:,1],
    'prob_2' : y_result_prob[:,2]
})

# Class 0일 확률 : y_result_prob[:, 0]
# Class 1일 확률 : y_result_prob[:, 1]
# Class 2일 확률 : y_result_prob[:, 2]

print(result_prob[:5])    

[2 2 2 0 1]
[[0.   0.   1.  ]
 [0.1  0.11 0.79]
 [0.   0.07 0.93]
 [0.97 0.03 0.  ]
 [0.04 0.86 0.1 ]]
   result  prob_0  prob_1  prob_2
0       2    0.00    0.00    1.00
1       2    0.10    0.11    0.79
2       2    0.00    0.07    0.93
3       0    0.97    0.03    0.00
4       1    0.04    0.86    0.10
